In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV

# Mount Drive, read file
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/aML/airbnb.csv')

Mounted at /content/drive


In [ ]:
# One-hot Encoding for city
df_encoded = pd.get_dummies(df, columns=['city'], prefix='city')

# Binary Encoding for time_period
df_encoded['is_weekend'] = (df_encoded['time_period'] == 'weekends').astype(int)

# Create combined stratification column
df_encoded['strat_column'] = df['city'] + '_' + df['time_period']

#Define features and target
X = df_encoded.drop(['price', 'log_price', 'time_period', 'strat_column'], axis=1)
y = df_encoded['log_price']

#Stratification split
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=df_encoded['strat_column']
)

In [ ]:
rfr = RandomForestRegressor(n_estimators=100, random_state=42)
rfr.fit(X_train, y_train)


print("Score of non-scaled Regressor on training set: {:.3f}".format(rfr.score(X_train, y_train)))
print("Score non-scaled Regressor on test set: {:.3f}".format(rfr.score(X_test, y_test)))

Score of non-scaled Regressor on training set: 0.978
Score non-scaled Regressor on test set: 0.850


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


numerical_cols = ['person_capacity', 'cleanliness_rating',
                 'guest_satisfaction_overall', 'city_dist', 'metro_dist']


numeric_transformer = StandardScaler()

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_cols)
    ],
    remainder='passthrough'
)

pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('rf', RandomForestRegressor(random_state=42))
])

param_grid = {
    'preprocessor__num': [StandardScaler(), 'passthrough'],
    'rf__n_estimators': [100, 200],
    'rf__max_depth': [None, 10, 20, 30],
    'rf__min_samples_split': [2, 5, 10],
    'rf__min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    cv=5,
    scoring='r2',
    return_train_score=True,
    verbose=3
)

grid_search.fit(X_train, y_train)

print("Best hyperparameter settings found: ", grid_search.best_params_)

best_model_rf = grid_search.best_estimator_

Fitting 5 folds for each of 144 candidates, totalling 720 fits
[CV 1/5] END preprocessor__num=StandardScaler(), rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=100;, score=(train=0.974, test=0.810) total time=  20.9s
[CV 2/5] END preprocessor__num=StandardScaler(), rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=100;, score=(train=0.974, test=0.812) total time=  20.3s
[CV 3/5] END preprocessor__num=StandardScaler(), rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=100;, score=(train=0.974, test=0.819) total time=  20.1s
[CV 4/5] END preprocessor__num=StandardScaler(), rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=100;, score=(train=0.974, test=0.824) total time=  20.2s
[CV 5/5] END preprocessor__num=StandardScaler(), rf__max_depth=None, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=100;, score=(train=0.974, test=0.8

In [ ]:
print("Best hyperparameter settings found: ", grid_search.best_params_)

best_model_rf = grid_search.best_estimator_

Best hyperparameter settings found:  {'preprocessor__num': StandardScaler(), 'rf__max_depth': None, 'rf__min_samples_leaf': 1, 'rf__min_samples_split': 2, 'rf__n_estimators': 200}
